In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import warnings
from datetime import datetime
import lightgbm as lgb

warnings.filterwarnings('ignore')

In [ ]:
dtype={
    1:'category',
    2:"float32",
    4:"uint16",
    5:"int32",
    6:"int32",
    7:"uint32"
}
from tqdm import tqdm
def get_df(f):
    files = []
    for chunk in tqdm(pd.read_csv(f, sep='\|', header=None, usecols=[1, 2, 4, 5, 6, 7], chunksize=1000000, dtype=dtype)):
        files.append(chunk)
    return files
chunk = get_df('E:\cp_rawdata_0509.txt')

df = pd.concat(chunk,axis=0)
del chunk

In [ ]:
col_names = ['uin', 'kill_time', 'index', 'deltaX', 'deltaY', 'button']
df.columns = col_names
df['uin'] = df['uin'].astype('category')
df.info()

In [ ]:
df_ = pd.read_csv('label.csv')
pos_uin = list(df_[df_['label'] == 1]['uin'])
neg_uin = list(df_[df_['label'] == 0]['uin'])
sample_uin = pos_uin + neg_uin
df = df[df['uin'].isin(sample_uin)]
df.info()

In [ ]:
df.sort_values(['uin', 'kill_time', 'index'], inplace=True)
df.to_pickle('train_total_.pkl')

In [ ]:
black = pd.read_csv('训练集black_uin.txt', sep='\|', header=None)
white = pd.read_csv('训练集white_uin.txt', sep='\|', header=None)
black.columns = ['uin']
white.columns = ['uin']
black['label'] = 1
white['label'] = 0
label_map = pd.concat([black, white], axis=0)
label_map = dict(zip(label_map['uin'], label_map['label']))

df_ = df.drop_duplicates('uin')
df_.drop(['kill_time', 'index', 'deltaX', 'deltaY', 'button'], axis=1, inplace=True)
df_ = df_[~df_['label'].isnull()]
df_.to_csv('label.csv', index=False)